# LangChain Lab: Exploring the Power of AI

Objectives:
- Install and configure a working environment for LangChain.
- Use an LLM (via LangChain) with prompt templates and chains.
- Explore agents and tools.
- Complete exercises.

## 0) Quick notes before you start

- You will need an OpenAI API key (or another provider) to run the LLM examples.
- Set `OPENAI_API_KEY` in your environment before running cells that call the LLM.
- This notebook uses `langchain` APIs; install the package with the cell below.

In [25]:

%pip install langchain openai tiktoken faiss-cpu chromadb langchain-openai langchain-tavily


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [26]:
import sys
print('Python', sys.version)

Python 3.11.11 (main, Dec  3 2024, 17:20:40) [Clang 16.0.0 (clang-1600.0.26.4)]


## 1) Simple LLM usage with LangChain

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "API_KEY_HERE"

OPENAI_MODEL = "gpt-5-mini"

In [28]:
# import logging
# logging.basicConfig(level=logging.INFO)

# from langchain_core.globals import set_debug
# set_debug(True)

In [29]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI

prompt_template = "Tell me a {adjective} joke"
prompt = PromptTemplate(
    input_variables=["adjective"], template=prompt_template
)
llm = OpenAI()
chain = prompt | llm | StrOutputParser()

response = chain.invoke("funny")

print(type(chain), type(llm), type(response))
print(response)

<class 'langchain_core.runnables.base.RunnableSequence'> <class 'langchain_openai.llms.base.OpenAI'> <class 'str'>
.

Why don't scientists trust atoms?

Because they make up everything.


In [30]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate


llm = ChatOpenAI(model=OPENAI_MODEL, temperature=0.3)

prompt = ChatPromptTemplate.from_template(
    "You are a math tutor. Explain the concept of {topic} clearly."
)

formatted_prompt = prompt.format_messages(topic="Agentic AI")

response = llm.invoke(formatted_prompt)

print(type(llm), type(response))
print(response.content)


<class 'langchain_openai.chat_models.base.ChatOpenAI'> <class 'langchain_core.messages.ai.AIMessage'>
Short answer
Agentic AI is any artificial system that acts in the world (real or simulated) to pursue goals over time, choosing actions based on perception, models or learning, and taking feedback into account. It is “agent‑like” — autonomous, goal‑directed, and capable of planning and adapting.

Why the term matters (intuitively)
- A tool answers once and stops (e.g., a calculator returns a result).  
- An agent continuously senses, decides, and acts to change its situation toward an objective (e.g., a vacuum robot that moves around, avoids obstacles, and tries to clean a room until it succeeds or its battery is low).  

Key components of an agentic AI
- Environment: what the agent perceives and acts upon.  
- State (or observations): information the agent receives.  
- Actions: choices the agent can make.  
- Policy / Decision rule: how the agent picks actions from states.  
- Object

In [31]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

class Country(BaseModel):
    capital: str = Field(description="capital of the country")
    name: str = Field(description="name of the country")

PROMPT_COUNTRY_INFO = """
Provide information about {country}.
{format_instructions}
"""

parser = PydanticOutputParser(pydantic_object=Country)
llm = ChatOpenAI(model=OPENAI_MODEL, temperature=0.3)

message = HumanMessagePromptTemplate.from_template(
    template=PROMPT_COUNTRY_INFO,
)
chat_prompt = ChatPromptTemplate.from_messages([message])

# Format the prompt with the country and parser instructions
chat_prompt_with_values = chat_prompt.format_prompt(
    country="Belgium",
    format_instructions=parser.get_format_instructions()
)

print("Input to the model")
for msg in chat_prompt_with_values.messages:
    print(f"{msg.type}: {msg.content}")

output = llm.invoke(chat_prompt_with_values.to_messages());

country = parser.parse(output.content)

print("-----------------------------------")
print("Output of the model: \n")
print(output.content)
print(f"The capital of {country.name} is {country.capital}.")


Input to the model
human: 
Provide information about Belgium.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"capital": {"description": "capital of the country", "title": "Capital", "type": "string"}, "name": {"description": "name of the country", "title": "Name", "type": "string"}}, "required": ["capital", "name"]}
```

-----------------------------------
Output of the model: 

{"name":"Belgium","capital":"Brussels"}
The capital of Belgium is Brussels.


In [32]:
from pydantic import BaseModel, Field

model = ChatOpenAI(model=OPENAI_MODEL, temperature=0.3)

class Movie(BaseModel):
    """A movie with details."""
    title: str = Field(..., description="The title of the movie")
    year: int = Field(..., description="The year the movie was released")
    director: str = Field(..., description="The director of the movie")
    rating: float = Field(..., description="The movie's rating out of 10")

model_with_structure = model.with_structured_output(Movie)
response = model_with_structure.invoke("Provide details about the movie Inception")
print(response)

title='Inception' year=2010 director='Christopher Nolan' rating=8.8


## 2) Agents: LLMs + Tools

Agents let a language model orchestrate tools (e.g., python REPL, search, calculator). Below is a minimal demonstration using LangChain's agent framework.

https://docs.langchain.com/oss/python/langchain/agents

In [33]:

from langchain.agents import create_agent

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

agent = create_agent(
    model=OPENAI_MODEL,
    tools=[get_weather],
    system_prompt="You are a helpful assistant",
)

response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

print(type(agent))
for msg in response["messages"]:
    msg.pretty_print()



<class 'langgraph.graph.state.CompiledStateGraph'>
================================ Human Message =================================

what is the weather in sf
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_Ujz5r5H6IoOEW7tt8hCbPvwB)
 Call ID: call_Ujz5r5H6IoOEW7tt8hCbPvwB
  Args:
    city: San Francisco, CA
================================= Tool Message =================================
Name: get_weather

It's always sunny in San Francisco, CA!
================================== Ai Message ==================================

For San Francisco (SF): "It's always sunny in San Francisco, CA!" 

Would you like current temperature, hourly forecast, or weather for a different location?


In [34]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=OPENAI_MODEL, temperature=0.3)

response = llm.invoke("Who are the professors in Raikes")

print(response.content)


Do you mean a specific place called "Raikes"? A few possibilities:

- The Raikes School (University of Nebraska–Lincoln)
- Raikes Hall / Raikes Building at a particular university (e.g., houses certain departments)
- Raikes Foundation (not a school — usually not described as having “professors”)
- Something else (a high school or program named Raikes)

Which one do you mean? If you tell me the institution or share a link, I can list the current professors (or give their profiles/contacts) for that Raikes program.


In [35]:
from langchain.agents import create_agent
import os
from langchain_tavily import TavilySearch

os.environ["TAVILY_API_KEY"] = "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"

tavily_search_tool = TavilySearch(
    max_results=5,
    topic="general",
    include_answer=True
)

agent = create_agent(
    model=OPENAI_MODEL,
    tools=[tavily_search_tool],
    system_prompt="You are a helpful assistant",
)

user_input = "Who are the professors in Raikes"
response = agent.invoke(
    {
        "messages": [
            {"role": "user", "content": user_input}
        ]
    }
)

for msg in response["messages"]:
    msg.pretty_print()


================================ Human Message =================================

Who are the professors in Raikes
================================== Ai Message ==================================

Which "Raikes" do you mean? A few possibilities:

- Raikes School (School of Computer Science & Management) at University of Nebraska–Lincoln  
- Raikes Foundation (charity / leadership programs)  
- Raikes Hall or another Raikes building at a specific university

Tell me which one (and whether you want current faculty only, titles/contact info, or research areas), and I’ll pull the list.


In [36]:

from langchain.agents import create_agent

#TODO: create an addition tool to give the model the ability to answer this question

agent = create_agent(
    model="gpt-3.5-turbo",
    tools=[],
    system_prompt="You are a helpful assistant",
)

user_input = "What is 13781 times 2394" # answer should be 32991714, model should get this wrong without a tool
response = agent.invoke(
    {"messages": [{"role": "user", "content": user_input}]}
)

for msg in response["messages"]:
        msg.pretty_print()



================================ Human Message =================================

What is 13781 times 2394
================================== Ai Message ==================================

13781 times 2394 equals 32,995,214.


In [37]:

from langchain.agents import create_agent

def your_tool(foo: str, bar: int) -> str:
    """
        Write the function docs here. They are important!
        This is how the model knows what the function does
        Make sure to include the types in the function definition.
        That is how the model knows what to pass in as arguements
    """
    # TODO: Implement tool
    return None

agent = create_agent(
    model=OPENAI_MODEL,
    tools=[your_tool],
    system_prompt="You are a helpful assistant",
)

user_input = "ask the model something here"
response = agent.invoke(
    {"messages": [{"role": "user", "content": user_input}]}
)

for msg in response["messages"]:
    msg.pretty_print()



================================ Human Message =================================

ask the model something here
================================== Ai Message ==================================

Do you mean you want me to formulate a question to send to another AI/model? If so, please tell me:

1. Topic or domain (coding, math, writing, product design, debugging, general knowledge, etc.).
2. Question type (open-ended, multiple choice, step-by-step problem, short answer).
3. Difficulty or detail level (beginner, intermediate, expert).
4. Any constraints or context to include (code language, word limit, facts/assumptions to provide).
5. Whether you want an ideal answer or rubric along with the question.

If you’re not sure, here are example questions I can use—pick one or tell me which style you want:

- Coding (debugging): "Here is a 30-line Python function that reverses words but produces incorrect output for some inputs. Identify the bug, explain why it fails, and provide a corrected im

In [ ]:
from pydantic import BaseModel
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy
from langchain.agents.structured_output import ProviderStrategy


class ContactInfo(BaseModel):
    name: str
    email: str
    phone: str

agent = create_agent(
    model=OPENAI_MODEL,
    tools=[],
    response_format=ToolStrategy(ContactInfo) #Change ToolStrategy with ProviderStrategy to see how output differs
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567"}]
})

for msg in result["messages"]:
    msg.pretty_print()

print("\n\nStructured response from agent:")
print(result["structured_response"])


<class 'langgraph.graph.state.CompiledStateGraph'>
================================ Human Message =================================

Extract contact info from: John Doe, john@example.com, (555) 123-4567
================================== Ai Message ==================================
Tool Calls:
  ContactInfo (call_QBAncp7FetcPij306ATB6ePf)
 Call ID: call_QBAncp7FetcPij306ATB6ePf
  Args:
    name: John Doe
    email: john@example.com
    phone: (555) 123-4567
================================= Tool Message =================================
Name: ContactInfo

Returning structured response: name='John Doe' email='john@example.com' phone='(555) 123-4567'


Structured response from agent:
name='John Doe' email='john@example.com' phone='(555) 123-4567'


In [ ]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver

# === Mock tools ===
def send_email_tool(recipient: str, subject: str, body: str) -> str:
    """Pretend to send an email."""
    return f"✅ Email sent to {recipient} with subject '{subject}'."

def read_email_tool() -> str:
    """Pretend to read an email."""
    return "📧 You have 2 unread emails from Alice and Bob."

tools = [send_email_tool, read_email_tool]

# === Create the agent ===
agent = create_agent(
    model=OPENAI_MODEL,
    tools=tools,
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "send_email_tool": {
                    "allowed_decisions": ["approve", "edit", "reject"],
                },
                "read_email_tool": False,  # auto-approve reads
            }
        ),
    ],
)

import random
thread_id = random.randint(1, 1000000)

result = agent.invoke({
    "messages": [
        {"role": "user", "content": "Send an email to alice@example.com saying I’ll be late to the meeting."}
    ]
}, {"configurable": {"thread_id": thread_id}})

last_msg = result["messages"][-1]
if hasattr(last_msg, "tool_calls") and last_msg.tool_calls:
    tool_call = last_msg.tool_calls[0]
    tool_name = tool_call["name"]
    tool_args = tool_call["args"]

    print(f"\nProposed tool call: {tool_name}")
    for k, v in tool_args.items():
        print(f"  {k}: {v}")

    decision = input("\nApprove or reject? ").strip().lower()

    if decision == "approve":
        tool_func = next(t for t in tools if t.__name__ == tool_name)
        output = tool_func(**tool_args)
        followup = agent.invoke({
            "messages": [
                *result["messages"],
                {"role": "tool", "name": tool_name, "content": output, "tool_call_id": tool_call["id"]},
            ]
        }, {"configurable": {"thread_id": thread_id}})
        for msg in followup["messages"]:
            msg.pretty_print()
    else:
        print("❌ Tool call rejected.")

else:
    for msg in result["messages"]:
        msg.pretty_print()
